### Sample plots to popular tabbed block components

In [ ]:
#| default_exp charts

In [47]:
#| export
import numpy as np
import pandas as pd
import altair as alt
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

Example use of a `toml` file might look like this.
> secrets.toml   

```
[api]
key = "YOUR_API_KEY_HERE"

[database]
username = "DB_USERNAME"
password = "DB_PASSWORD"
``````
Remember to add it to your `.gitignore` file ⚠️

In [9]:
#hide
from nbdev.showdoc import *
import toml
s = toml.load("../.gradio/secrets.toml", _dict=dict)
s['data']

{'luke': 'test'}

# 1. Pull Sample Data

In [24]:
#| export

def get_sample(verbose=True):
    """
    Sample Sales Data, Order Info, Sales, Customer, Shipping, etc., 
    Used for Segmentation, Customer Analytics, Clustering and More. 
        - Taken from Kaggle (www.kaggle.com/datasets/kyanyoga/sample-sales-data)
    """
    data = pd.read_csv('../data/sales_data_sample.csv', encoding=('ISO-8859-1'))

    data['ORDERDATE'] = pd.to_datetime(data['ORDERDATE'])

    # Removing all the columns not revelant for this analysis to avoid confusion
    data.drop(['ORDERLINENUMBER','STATUS','PRODUCTCODE','PHONE','STATE',
           'POSTALCODE', 'TERRITORY', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME'], axis=1)

    if verbose:
        print(data.shape)
        display(data.head())
    return data

In [25]:
data = get_sample()

(2823, 25)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [41]:
# Extract the month and year from the ORDERDATE for grouping
data['YearMonth'] = data['ORDERDATE'].dt.to_period('M')

# Group by YearMonth and sum the SALES
monthly_sales_alt = data.groupby('YearMonth')['SALES'].sum().reset_index()

# Convert the 'YearMonth' back to string for Altair to handle it properly
monthly_sales_alt['YearMonth'] = monthly_sales_alt['YearMonth'].astype(str)

In [45]:

# Create the Altair chart
chart = alt.Chart(monthly_sales_alt).mark_bar(color='cyan').encode(
    x=alt.X('YearMonth:O', title='Month', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('SALES:Q', title='Total Sales'),
    tooltip=['YearMonth', 'SALES']
).properties(
    title='Monthly Sales Over Time',
    width=800,
    height=400
)

# Add text labels on top of the bars
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-10  # Nudge text upwards
).encode(
    text='SALES:Q'
)

# Combine the chart with bars and text labels
final_chart = (chart + text)

# To display the chart, you'd typically use:
# final_chart.show()
final_chart

alt.LayerChart(...)

In [49]:
# export 

def human_readify(n):
    """Convert large numbers into a human-readable format."""
    millnames = ['', ' K', ' M', ' B']
    n = float(n)
    millidx = max(0, min(len(millnames)-1, 
                         int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    return '{:.1f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

In [55]:
# First, we need to regenerate the 'YearMonth' column and group the sales by month

# Extract the month and year from the ORDERDATE for grouping
data['YearMonth'] = data['ORDERDATE'].dt.to_period('M')

# Group by YearMonth and sum the SALES
monthly_sales_alt = data.groupby('YearMonth')['SALES'].sum().reset_index()

# Convert the 'YearMonth' back to string for Altair to handle it properly
monthly_sales_alt['YearMonth'] = monthly_sales_alt['YearMonth'].astype(str)

# Convert the SALES column to a human-readable format
monthly_sales_alt['ReadableSales'] = monthly_sales_alt['SALES'].apply(human_readable_number)

# Create the Altair chart with improved aesthetics
chart = alt.Chart(monthly_sales_alt).mark_bar(color='lightblue').encode(
    x=alt.X('YearMonth:O', title='Month', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('SALES:Q', title='Total Sales', axis=alt.Axis(format=".1s")),
    tooltip=['YearMonth', 'ReadableSales']
).properties(
    title='Monthly Sales Over Time',
    width=800,
    height=400
)

# Add text labels on top of the bars
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-10  # Nudge text upwards
).encode(
    text='ReadableSales:O'
)

# Combine the chart with bars and text labels
final_chart = (chart + text)

final_chart


alt.LayerChart(...)

In [59]:
# First, we need to regenerate the 'YearMonth' column and group the sales by month

# Extract the month and year from the ORDERDATE for grouping
data['YearMonth'] = data['ORDERDATE'].dt.to_period('M')

# Group by YearMonth and sum the SALES
monthly_sales_alt = data.groupby('YearMonth')['SALES'].sum().reset_index()

# Convert the 'YearMonth' back to string for Altair to handle it properly
monthly_sales_alt['YearMonth'] = monthly_sales_alt['YearMonth'].astype(str)

# Convert the SALES column to a human-readable format
monthly_sales_alt['ReadableSales'] = monthly_sales_alt['SALES'].apply(human_readable_number)

# Create the Altair chart with improved aesthetics
chart = alt.Chart(monthly_sales_alt).mark_bar(color='lightblue').encode(
    x=alt.X('YearMonth:O', title='Month', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('SALES:Q', title='Total Sales', axis=alt.Axis(format=".1s")),
    tooltip=['YearMonth', 'ReadableSales']
).properties(
    title='Monthly Sales Over Time',
    width=800,
    height=400
)

# Determine the top 3 sales values
top_3_sales = monthly_sales_alt['SALES'].nlargest(3).tolist()

# Get the first and last YearMonth values
first_month = monthly_sales_alt['YearMonth'].iloc[0]
last_month = monthly_sales_alt['YearMonth'].iloc[-1]

# Get the sales values for the first and last months
first_month_sales = monthly_sales_alt[monthly_sales_alt['YearMonth'] == first_month]['SALES'].iloc[0]
last_month_sales = monthly_sales_alt[monthly_sales_alt['YearMonth'] == last_month]['SALES'].iloc[0]

# Add text labels on top of the bars conditionally
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-10  # Nudge text upwards
).encode(
    text=alt.condition(
        (alt.datum.SALES == top_3_sales[0]) | 
        (alt.datum.SALES == top_3_sales[1]) | 
        (alt.datum.SALES == top_3_sales[2]) |
        ((alt.datum.YearMonth == first_month) & (alt.datum.SALES == first_month_sales)) | 
        ((alt.datum.YearMonth == last_month) & (alt.datum.SALES == last_month_sales)),
        'ReadableSales:O',
        alt.value('')  # This means no text for other bars
    )
)

# Combine the chart with bars and text labels
final_chart = (chart + text)
final_chart

alt.LayerChart(...)

In [50]:
#| hide
import nbdev; nbdev.nbdev_export()